In [1]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [2]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [6]:
query = """
WITH
info AS (
SELECT 
    MAX(fbi.dim_date) AS dim_date, 
    year_month_char,
    dim_stock_unit 
FROM dpr_cross_business.fact_cross_business_insights fbi
INNER JOIN dpr_shared.dim_date d ON d.date_id = fbi.dim_date
WHERE margin_mtd <> 0
 AND full_date >= DATE_TRUNC('month', date(getdate()) - interval '0 month')
GROUP BY 2,3
)

SELECT DISTINCT
    s.identifier_value AS region,
    su.sku,
    su.card_id,
    i.year_month_char AS month,
    fbi.margin_mtd AS margin
FROM dpr_cross_business.fact_cross_business_insights fbi
INNER JOIN dpr_shared.dim_stock_unit su ON su.stock_unit_id = fbi.dim_stock_unit
INNER JOIN dpr_shared.dim_site s ON s.site_id = fbi.dim_site
INNER JOIN info i ON i.dim_date = fbi.dim_date AND i.dim_stock_unit = fbi.dim_stock_unit
"""
df_margin = run_read_dwd_query(query)

In [7]:
df_hooks = get_fresh_query_result("https://internal-redash.federate.frubana.com/",108977,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [8]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
city = 'VCP'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')

In [47]:
def filter_customers(city, recency_day_low=7, recency_day_top=30,  kam=100, graduate=100, msgt=('ALL'), sgt=('ALL')):
    """
    This function filters the customers that have completed the specified criteria:
    city: The city that the customer has been registered -> i.e 'BOG'
    recency_day: The limit of days that the customer has completed an order -> 30 means 30 days or less
    kam: If the customer is a KAM or No -> {100: ALL, 1: True, 0: False}
    graduate: If the customer is GRADUATED or No -> {100: ALL, 1: True, 0: False}
    msgt: The microsegment of the customer, this is different based on the city
        - If you want to bring all should put as a tuple ('ALL')
        - If you want to bring one msgt then i.e ('Fonda')
        - If you want to bring more than one msgt then i.e ('Fonda','Corrientazo')
    sgt: The segment of the customer
        - If you want to bring all should put as a tuple ('ALL')
        - If you want to bring one msgt then i.e ('Restaurante')
        - If you want to bring more than one msgt then i.e ('Comercio','Restaurante')
    """
    import pandas as pd
    from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

    if isinstance(msgt, str):
        msgt = (msgt,'a')  # Convert the string to a tuple with a single element
    if isinstance(sgt, str):
        sgt = (sgt,'a')  # Convert the string to a tuple with a single element

    query = """
    SELECT
        source_id AS customer_id,
        is_kam_audited,
        birthday,
        last_order_date,
        recency_days,
        registered_city AS region,
        salesperson_mail,
        microsegment_name AS microsegment,
        business_type AS segment,
        is_graduated
    FROM dpr_customer.obt_customer

    WHERE recency_days <= {days}
    AND recency_days >= {days2}
    ----------------------------------
    ----------------------------------
    AND 
    CASE
    WHEN '{city}' = 'ALL' THEN 1=1
    ELSE registered_city = '{city}'
    END
    ----------------------------------
    ----------------------------------
    AND
    CASE
    WHEN {is_kam} = 100 THEN 1=1
    ELSE is_kam_audited = {is_kam}
    END
    ----------------------------------
    ----------------------------------
    AND
    CASE
    WHEN {graduated} = 100 THEN 1=1
    ELSE is_graduated = {graduated}
    END
    ----------------------------------
    ----------------------------------
    AND 
    CASE
    WHEN 'ALL' IN {microsegment} THEN 1=1
    ELSE microsegment_name IN {microsegment}
    END
    ----------------------------------
    ----------------------------------
    AND
    CASE
    WHEN 'ALL' IN {segment} THEN 1=1
    ELSE business_type IN {segment}
    END
    """.format(days=recency_day_top,days2=recency_day_low,city=city,is_kam=kam,graduated=graduate,microsegment=msgt,segment=sgt)

    return run_read_dwd_query(query)

In [10]:
def info_sells(city, sells_days_back_start=30, days_end=-1, customer=None):    
    """
    This returns a df with the information of the whole sells order by order based on the restrictions
    city: The city that the customer has been registered -> i.e 'BOG'
    sells_days_back_start: Since when we want to see the orders information
    days_end: Until when we want to see the orders information
    """    
    import pandas as pd
    from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
    import time
    import datetime
    
    sells_date_inicio = (datetime.datetime.today() - datetime.timedelta(days=sells_days_back_start)).strftime('%Y-%m-%d') #1 months
    sells_date_fin = (datetime.datetime.today() - datetime.timedelta(days=days_end)).strftime('%Y-%m-%d') #1 months
    
    tasas = {'BOG': 3776,
            'BAQ': 3776,
            'MDE': 3776,
            'CMX': 19.65,
            'GDL': 19.65,
            'PBC': 19.65,
            'SPO': 4.75,
            'BHZ': 4.75,
            'CWB': 4.75,
            'VCP': 4.75}

    if not isinstance(customer, list):
        raise TypeError("Customer parameter must be a list.")
    elif customer is None:    
        query = """
        SELECT 
            s.identifier_value AS region_code,
            DATE(fs.order_submitted_date) AS submit_date,
            fs.order_close_date AS close_date,
            fs.order_id,
            dc.source_id AS customer_id,
            cat.parent_description as category,
            cat.description AS subcategory,
            dp.card_id,
            dp.source_id AS sku_id,
            dp.card_description AS product_name,
            fs.product_quantity_x_step_unit AS cant,
            fs.gmv_pxq_local/{tasa_cambio} AS gmv_usd,
            COALESCE(fsd.product_discount/{tasa_cambio},0) AS discount_applied,
            fsd.adjustment_reason,
            COALESCE(bo.source_id,NULL) AS offer_id
            
        FROM dpr_sales.fact_sales                   fs
        INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
        INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
        INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
        INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
        LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
        LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     

        WHERE 
            fs.gmv_enabled = TRUE
            AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
            AND fs.fb_order_status_id  IN (1,6,7,8)
            AND fs.is_deleted = FALSE
            AND fs.dim_status = 1
            AND dp.is_slot = 'false'
            --AND fsd.is_enabled = true
            AND s.identifier_value = '{ciudad}'
            AND DATE(fs.order_submitted_date) between '{Fecha_start}' AND '{Fecha_end}'
    """.format(tasa_cambio = tasas[city], ciudad = city, Fecha_start = sells_date_inicio, Fecha_end = sells_date_fin)
    else:
        query = """
        SELECT 
            s.identifier_value AS region_code,
            DATE(fs.order_submitted_date) AS submit_date,
            fs.order_close_date AS close_date,
            fs.order_id,
            dc.source_id AS customer_id,
            cat.parent_description as category,
            cat.description AS subcategory,
            dp.card_id,
            dp.source_id AS sku_id,
            dp.card_description AS product_name,
            fs.product_quantity_x_step_unit AS cant,
            fs.gmv_pxq_local/{tasa_cambio} AS gmv_usd,
            COALESCE(fsd.product_discount/{tasa_cambio},0) AS discount_applied,
            fsd.adjustment_reason,
            COALESCE(bo.source_id,NULL) AS offer_id
            
        FROM dpr_sales.fact_sales                   fs
        INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
        INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
        INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
        INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
        LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
        LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     

        WHERE 
            fs.gmv_enabled = TRUE
            AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
            AND fs.fb_order_status_id  IN (1,6,7,8)
            AND fs.is_deleted = FALSE
            AND fs.dim_status = 1
            AND dp.is_slot = 'false'
            --AND fsd.is_enabled = true
            AND s.identifier_value = '{ciudad}'
            AND DATE(fs.order_submitted_date) between '{Fecha_start}' AND '{Fecha_end}'
            AND dc.source_id IN {clientes}
    """.format(tasa_cambio = tasas[city], ciudad = city, Fecha_start = sells_date_inicio, Fecha_end = sells_date_fin,clientes=tuple(set(customer)))   
    return run_read_dwd_query(query)

In [11]:
group = ('ALL') #the microsegments you want

In [48]:
#getting the clients that i want
df_customers = filter_customers(city,7,30,100,100,group,('Restaurante'))#.customer_id.unique() #,'Taquería','Comida Rápida','Panadería/Pastelería'

In [50]:
len(df_customers)

1485

### Loading COSTUMERS INFO

In [49]:
data_ventas = info_sells(city, sells_days_back_start=60, days_end=-1, customer=df_customers.customer_id.unique().tolist())

In [21]:
data_ventas_IQ = data_ventas.groupby(by=['region_code','customer_id']).agg({"gmv_usd":np.sum, "order_id":pd.Series.nunique}).reset_index()
data_ventas_IQ["AOV"] = data_ventas.gmv_usd/data_ventas.order_id

In [51]:
from sklearn.model_selection import train_test_split 

def create_groups(offer):
    
    customers = offer.customer_id.unique()
    print("Total length: ", len(customers))
    customer_control, customer_test = train_test_split(customers, test_size = 0.85)
    
    alpha=0.01
    aux=0
    while abs(1-(offer[offer.customer_id.isin(customer_control)]['AOV'].mean()/offer[offer.customer_id.isin(customer_test)]['AOV'].mean())) > alpha:
        customer_control, customer_test = train_test_split(customers, test_size = 0.85)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
        
    return customer_control.tolist(), customer_test.tolist()

In [28]:
"""Here we should complete our data for each SH, important if we want to submit a subcategory SH offer, in SKUS we should put the skus into a list as this [[1,2,3]]"""
card_id = [325412,325413]
max_uses = [30,30]
max_orders = [2,2]
discount = [8,5]

In [52]:
offer_name = []
global_segment_list = [] #//for procesing
hooks = []

control, test = create_groups(data_ventas_IQ)

for sku in card_id:
    name = frida_products.loc[frida_products.id_tarjeta == sku,'card_name'].values[0]
    offer_name.append("SH_CUORTO_"+todays_date+"_"+city+"_"+name)
    

    d = {'name':"SH_CUORTO_"+todays_date+"_"+city+"_"+name, 'customersIds': list(test),"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(1))).strftime('%Y-%m-%d')}
    e = {'name':"SH_CUORTO_"+todays_date+"_"+city+"_"+name+"_CONTROL", 'customersIds': list(control),"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(1))).strftime('%Y-%m-%d')}
    
    global_segment_list.append(d)
    global_segment_list.append(e)
    
    if df_hooks.loc[df_hooks.addl_product_id == sku].value_discount.max():
        hooks.append(df_hooks.loc[df_hooks.addl_product_id == sku].value_discount.max())
    else:
        hooks.append(0)

Total length:  2839


In [33]:
offer_name2 = []
for i, name in enumerate(offer_name):
    # Retrieve rows that match the condition
    matching_rows = df_hooks.loc[df_hooks.addl_product_id == card_id[i], ["value_discount"]]
    
    # Check if matching_rows is empty
    if matching_rows.empty:
        default_value = 0
    else:
        # Get the unique discount value from matching rows
        discount_value = matching_rows["value_discount"].unique()[0]
        
        # Calculate the updated value
        default_value = round(discount_value / discount[i], 2)
    
    # Update offer_name[i] with the calculated value
    offer_name2.append(offer_name[i] + "_HDCT_" + str(default_value))    
print(offer_name2)

['SH_CUORTO_2023-11-16_VCP_Óleo de Soja Vitaliv 900ml_HDCT_0.75', 'SH_CUORTO_2023-11-16_VCP_Leite Integral Shefa 1L_HDCT_0']


In [34]:
print(offer_name)
print(offer_name2)

['SH_CUORTO_2023-11-16_VCP_Óleo de Soja Vitaliv 900ml', 'SH_CUORTO_2023-11-16_VCP_Leite Integral Shefa 1L']
['SH_CUORTO_2023-11-16_VCP_Óleo de Soja Vitaliv 900ml_HDCT_0.75', 'SH_CUORTO_2023-11-16_VCP_Leite Integral Shefa 1L_HDCT_0']


In [35]:
csv_file = {} 
csv_file['offer_name'] = offer_name2
csv_file['offer_description'] = None
csv_file['discount'] = discount
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=0)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(1))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = max_uses
csv_file['max_uses_per_customer'] = max_orders
csv_file['customer_segment_id'] = [1]*len(offer_name) #[None]
csv_file['sku_id'] = [frida_products.loc[frida_products.id_tarjeta == sku,"skus"].values[0] for sku in card_id]
csv_file['segment_behaviour'] = 'INCLUDED'
csv_file['offer_type'] = 'SUPER_DISCOUNT'
csv_file['supplier_discount'] = 0

global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [38]:
import requests
import json

url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [39]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [40]:
sgt

[(501504, 'SH_CUORTO_2023-11-16_VCP_Óleo de Soja Vitaliv 900ml'),
 (501502, 'SH_CUORTO_2023-11-16_VCP_Óleo de Soja Vitaliv 900ml_CONTROL'),
 (501505, 'SH_CUORTO_2023-11-16_VCP_Leite Integral Shefa 1L'),
 (501503, 'SH_CUORTO_2023-11-16_VCP_Leite Integral Shefa 1L_CONTROL')]

In [41]:
for i,name in enumerate(global_offer_csv.offer_name):
    for j,z in sgt:
        if z==offer_name[i]:
            global_offer_csv.loc[[i], ('customer_segment_id')] = j
            break

In [42]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_CUORTO_2023-11-16_VCP_Óleo de Soja Vitaliv ...,None,8,true,16-11-2023,17-11-2023,30,2,501504,428399
1,SH_CUORTO_2023-11-16_VCP_Leite Integral Shefa ...,None,5,true,16-11-2023,17-11-2023,30,2,501505,428402


In [ ]:
# Add "group" column based on customer_id
df_customers['group'] = 'none'  # Initialize the group column with a default value

# Mark customers in the control list as 'control' in the 'group' column
df_customers.loc[df_customers['customer_id'].isin(control), 'group'] = 'control'

# Mark customers in the test list as 'test' in the 'group' column
df_customers.loc[df_customers['customer_id'].isin(test), 'group'] = 'test'

In [61]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'{city}/{todays_date}_SH_CUORTO_{city}_OFFER_csv.csv', index = False)
df_customers.to_csv(f'{city}/{todays_date}_SH_CUORTO_INFO_CUSTOMERS.csv', index=False)